Testing Goteborg

In [22]:
import networkx as nx
import os
import fiona
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, Point, Polygon
from shapely.ops import unary_union
import osmnx as ox
import matplotlib.pyplot as plt

Load the Network from the Geodatabase

In [4]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
temp_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step2_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2')
if not os.path.exists(step2_f):
    os.makedirs(step2_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')

In [ ]:
import fiona
import geopandas as gpd

# Define the path to the geodatabase and the layer name
gdb_path = 'path/to/your/geodatabase.gdb'
layer_name = 'your_network_layer'

# Load the network data using GeoPandas
gdf = gpd.read_file(gdb_path, layer=layer_name)

In [8]:
ctr_cd = 'SE'
pedestrianNWpath = networks_f + r"\NW_" + ctr_cd + ".gdb"
layers = fiona.listlayers(pedestrianNWpath)
print(layers)

['SE_Buff_PL', 'nw', 'Network_ND_Junctions', 'ND_7_DirtyAreas', 'ND_7_DirtyObjects', 'N_1_Desc', 'N_1_EdgeWeight', 'N_1_JunctionWeight', 'N_1_Props', 'N_1_Topology', 'N_1_TurnWeight']


In [11]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
outdata_f = os.path.join(indata_f, 'Processing', 'Test', 'Goteborg')
os.makedirs(outdata_f, exist_ok=True)

# Define engines
engines = {
    'fiona': {'engine': 'fiona'},
    'pyogrio': {'engine': 'pyogrio'},
    'pyogrio+arrow': {'engine': 'pyogrio', 'use_arrow': True}
          
}


In [12]:
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB1B2.shp')
# Read the GeoPackage file
uc = gpd.read_file(uc_file_path)
uc_city_code = 'GEOSTAT21_029'
uc_city = uc.query(f"HDENS_CLST=='{uc_city_code}'")

In [13]:
layer_name = 'nw'
gdf = gpd.read_file(pedestrianNWpath, layer=layer_name, engine='pyogrio', 
                    use_arrow=True, bbox= tuple(uc_city.total_bounds))

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\contextlib.py:119: RuntimeWarning: Cannot use P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Network data Regio\NW_SE.gdb\a0000000a.spx as the index depth(=1) is suspicious (it should rather be 2)
  return next(self.gen)


In [37]:
gdf = gdf.to_crs(epsg=4326)

In [52]:
# Initialize an empty graph
G = nx.Graph()

# Add edges to the graph
for idx, row in gdf.iterrows():
    geometry = row['geometry']
    
    # Check if geometry is a MultiLineString
    if isinstance(geometry, MultiLineString):
        # If MultiLineString, iterate over each LineString using `geoms` attribute
        for line in geometry.geoms:
            start_point = line.coords[0]
            end_point = line.coords[-1]
            G.add_edge(Point(start_point), Point(end_point), length=line.length)
    elif isinstance(geometry, LineString):
        # If LineString, directly add it to the graph
        start_point = geometry.coords[0]
        end_point = geometry.coords[-1]
        G.add_edge(Point(start_point), Point(end_point), length=geometry.length)

In [19]:
path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\OutputData\batch1_allprocessed\step2\SA_output_batch1_v0108_6_7_polygon.gdb'
center_points_gdf = gpd.read_file(path, layer='GEOSTAT21_029_GUA_pt_50m_nw_25m', engine='pyogrio', 
                    use_arrow=True, bbox= tuple(uc_city.total_bounds))


In [21]:
# Extract the Point geometries from the GeoDataFrame
center_points = [geom for geom in center_points_gdf.geometry if isinstance(geom, Point)]

In [23]:
def create_service_areas(graph, center_points, distance):
    service_area_polygons = []

    for point in center_points:
        # Find the nearest node to the current point
        center_node = ox.get_nearest_node(graph, (point.y, point.x))

        # Use NetworkX to find nodes within a certain distance
        subgraph = nx.ego_graph(graph, center_node, radius=distance, distance='length')
        
        # Get the edges and nodes of the subgraph
        nodes, edges = ox.graph_to_gdfs(subgraph)
        
        # Extract the coordinates of the nodes
        points = [Point(data['x'], data['y']) for node, data in subgraph.nodes(data=True)]
        
        # Create a convex hull polygon around these points (service area)
        if points:
            polygon = Polygon(points).convex_hull
            service_area_polygons.append(polygon)

    # Combine all service area polygons into one
    if service_area_polygons:
        combined_service_area = unary_union(service_area_polygons)
        return combined_service_area, service_area_polygons

    return None, []

In [25]:
def create_service_areas(graph, center_points, distance_value):
    service_area_polygons = []

    for point in center_points:
        # Find the nearest node to the current point using nearest_nodes
        center_node = distance.nearest_nodes(graph, point.x, point.y)

        # Use NetworkX to find nodes within a certain distance
        subgraph = nx.ego_graph(graph, center_node, radius=distance_value, distance='length')
        
        # Get the edges and nodes of the subgraph
        nodes, edges = ox.graph_to_gdfs(subgraph)
        
        # Extract the coordinates of the nodes
        points = [Point(data['x'], data['y']) for node, data in subgraph.nodes(data=True)]
        
        # Create a convex hull polygon around these points (service area)
        if points:
            polygon = Polygon(points).convex_hull
            service_area_polygons.append(polygon)

    # Combine all service area polygons into one
    if service_area_polygons:
        combined_service_area = unary_union(service_area_polygons)
        return combined_service_area, service_area_polygons

    return None, []

In [30]:
# Reproject the center points to WGS84 (EPSG:4326)
center_points_gdf = center_points_gdf.to_crs(epsg=4326)

# Extract the Point geometries
center_points = [geom for geom in center_points_gdf.geometry if isinstance(geom, Point)]

In [31]:
from osmnx.distance import nearest_nodes
# Define the service area distance (e.g., 500 meters)
distance_value = 500

# Function to create service areas
def create_service_areas(graph, center_points, distance_value):
    service_area_polygons = []

    for point in center_points:
        # Find the nearest node to the current point using nearest_nodes
        center_node = distance.nearest_nodes(graph, point.x, point.y)

        # Use NetworkX to find nodes within a certain distance
        subgraph = nx.ego_graph(graph, center_node, radius=distance_value, distance='length')
        
        # Get the edges and nodes of the subgraph
        nodes, edges = ox.graph_to_gdfs(subgraph)
        
        # Extract the coordinates of the nodes
        points = [Point(data['x'], data['y']) for node, data in subgraph.nodes(data=True)]
        
        # Create a convex hull polygon around these points (service area)
        if points:
            polygon = Polygon(points).convex_hull
            service_area_polygons.append(polygon)

    # Combine all service area polygons into one
    if service_area_polygons:
        combined_service_area = unary_union(service_area_polygons)
        return combined_service_area, service_area_polygons

    return None, []

# Create the combined service area and individual service areas
combined_service_area, service_area_polygons = create_service_areas(G, center_points, distance_value)

# Convert the combined service area to a GeoDataFrame
combined_service_area_gdf = gpd.GeoDataFrame([combined_service_area], columns=['geometry'])

# Plot the graph and combined service area
fig, ax = plt.subplots(figsize=(10, 10))
edges.plot(ax=ax, linewidth=1, edgecolor='black')
combined_service_area_gdf.boundary.plot(ax=ax, edgecolor='blue', linewidth=2)

plt.show()

AttributeError: 'int' object has no attribute 'nearest_nodes'

In [ ]:
orig = ox.nearest_nodes(G, 51.262336765, 7.143472955)
dest = ox.nearest_nodes(G, 51.2521799,7.1491453)

In [47]:
print(G.graph.get('crs')) 

EPSG:4326


In [46]:
G.graph['crs'] = 'EPSG:4326'

In [48]:
center_node =  ox.nearest_nodes(G, 11.989759108194786, 57.68219439583089)

KeyError: "None of [Index(['x', 'y'], dtype='object')] are in the [columns]"

In [49]:
import osmnx as ox
from osmnx.distance import nearest_nodes

# Load the network data
G = ox.graph_from_place('Piedmont, California, USA', network_type='drive')

# Set CRS if not present
if 'crs' not in G.graph:
    G.graph['crs'] = 'EPSG:4326'  # Replace with the correct CRS if different

# Verify CRS
print("Graph CRS:", G.graph.get('crs'))

# Convert graph to GeoDataFrames
nodes_df, _ = ox.graph_to_gdfs(G, nodes=True, edges=False, node_geometry=True, fill_edge_geometry=False)

# Extract 'x' and 'y' if necessary
if 'geometry' in nodes_df.columns:
    nodes_df['x'] = nodes_df['geometry'].x
    nodes_df['y'] = nodes_df['geometry'].y

# Define coordinates
x, y = 11.989759108194786, 57.68219439583089

# Find the nearest node
center_node = nearest_nodes(G, x, y)
print(f"The nearest node to the coordinates ({x}, {y}) is {center_node}.")


Graph CRS: epsg:4326


ValueError: too many values to unpack (expected 2)

In [51]:
import osmnx as ox
from osmnx.distance import nearest_nodes

# Load the network data
G = ox.graph_from_place('Piedmont, California, USA', network_type='drive')

# Ensure CRS is set
G.graph['crs'] = 'EPSG:4326'  # Replace with the correct CRS if different

# Convert graph to GeoDataFrames
nodes_df, edges_df = ox.graph_to_gdfs(G, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=False)

# Extract 'x' and 'y' from node_geometry if necessary
if 'geometry' in nodes_df.columns:
    nodes_df['x'] = nodes_df['geometry'].x
    nodes_df['y'] = nodes_df['geometry'].y

# Define coordinates
x, y = 11.989759108194786, 57.68219439583089

# Find the nearest node
center_node = nearest_nodes(G, x, y)
print(f"The nearest node to the coordinates ({x}, {y}) is {center_node}.")



The nearest node to the coordinates (11.989759108194786, 57.68219439583089) is 3905291938.


In [ ]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

# Step 1: Load Network Data
G = ox.graph_from_place('Piedmont, California, USA', network_type='drive')

# Set CRS if not present
G.graph['crs'] = 'EPSG:4326'

# Step 2: Load Points from GeoDataFrame
# Assuming `points_gdf` is your GeoDataFrame with points
points_gdf = gpd.read_file('path_to_your_points_file.geojson')  # Replace with your file

# Ensure points are in the same CRS as the graph
points_gdf = points_gdf.to_crs(G.graph['crs'])

# Step 3: Create Service Areas
def create_service_areas(G, points_gdf, distance_value):
    service_area_polygons = []
    
    for point in points_gdf.geometry:
        # Find the nearest node to the current point
        x, y = point.x, point.y
        center_node = ox.distance.nearest_nodes(G, x, y)
        
        # Create subgraph around the nearest node
        subgraph = nx.ego_graph(G, center_node, radius=distance_value, distance='length')
        
        # Convert subgraph to a GeoDataFrame
        nodes_df, edges_df = ox.graph_to_gdfs(subgraph, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=False)
        
        # Create a convex hull polygon around the nodes
        if not nodes_df.empty:
            polygons = nodes_df.geometry.unary_union.convex_hull
            service_area_polygons.append(polygons)
    
    # Combine all service areas into a single MultiPolygon
    combined_service_area = unary_union(service_area_polygons)
    
    return combined_service_area

# Define distance (in meters) for the service area
distance_value = 1000  # Example: 1000 meters

# Create service areas
combined_service_area = create_service_areas(G, points_gdf, distance_value)

# Step 4: Convert to GeoDataFrame
combined_service_area_gdf = gpd.GeoDataFrame(
    {'geometry': [combined_service_area]},
    crs=G.graph['crs']
)

# Save or plot the result
combined_service_area_gdf.to_file('path_to_output_service_areas.geojson', driver='GeoJSON')
combined_service_area_gdf.plot()


In [44]:
G.graph["crs"]

KeyError: 'crs'

In [40]:
from osmnx.distance import nearest_nodes 
for point in center_points:
    print(point.x, point.y)
    # Find the nearest node to the current point using nearest_nodes
    center_node = distance.nearest_nodes(G, point.x, point.y)

11.989759108194786 57.68219439583089


AttributeError: 'int' object has no attribute 'nearest_nodes'

In [32]:
center_points

[<POINT (11.99 57.682)>,
 <POINT (11.991 57.682)>,
 <POINT (11.991 57.681)>,
 <POINT (11.991 57.681)>,
 <POINT (11.99 57.681)>,
 <POINT (11.989 57.681)>,
 <POINT (11.989 57.682)>,
 <POINT (11.989 57.682)>,
 <POINT (11.99 57.682)>,
 <POINT (11.99 57.682)>]

In [24]:
# Define the service area distance (e.g., 500 meters)
distance = 500

# Create the combined service area and individual service areas
combined_service_area, service_area_polygons = create_service_areas(G, center_points, distance)

# Convert the combined service area to a GeoDataFrame
combined_service_area_gdf = gpd.GeoDataFrame([combined_service_area], columns=['geometry'])

# Plot the graph and combined service area
fig, ax = plt.subplots(figsize=(10, 10))
edges.plot(ax=ax, linewidth=1, edgecolor='black')
combined_service_area_gdf.boundary.plot(ax=ax, edgecolor='blue', linewidth=2)

plt.show()

AttributeError: module 'osmnx' has no attribute 'get_nearest_node'

In [ ]:


pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"

ctr_cd = 'SE'
pedestrianNWpath = networks_f + r"\NW_" + ctr_cd + ".gdb

pedestrianNWpath
pedestrianNWpath = networks_f + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"



# input parameters
pedestrian_path = networks_f
outPath = step2_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"
nbMeters = 400